# other stuff

In [ ]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import getpass
from utils import plot_image

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

In [ ]:
SHConfig.get_config_location()

In [ ]:
# if config not set up properly:
load_dotenv() 
config = SHConfig(use_defaults=True)
config.sh_client_id = os.getenv("CLIENT_ID")
config.sh_client_secret = os.getenv("CLIENT_SECRET")
config.sh_token_url = os.getenv("TOKEN_URL")
config.sh_base_url = os.getenv("BASE_URL")
config.save("cdse")

In [ ]:
#else: 
config = SHConfig("cdse")

In [ ]:
start_date = "2019-07-01"
end_date = "2019-07-31"
data_collection = "SENTINEL-2"
#aoi = "POLYGON((4.220581 50.958859,4.521264 50.953236,4.545977 50.906064,4.541858 50.802029,4.489685 50.763825,4.23843 50.767734,4.192435 50.806369,4.189689 50.907363,4.220581 50.958859))'"
aoi = "POLYGON((12 45.5,12 44.5,13 44.5,13 45.5,12 45.5))'"

In [ ]:
json = requests.get(
    f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}) and ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T00:00:00.000Z"
).json()
pd.DataFrame.from_dict(json["value"]).head(5)

In [ ]:
request_4_bands = SentinelHubRequest(
data_folder='../data',
    evalscript=evalscript_true_color,
    input_data=[
                   
       SentinelHubRequest.input_data(
           data_collection=DataCollection.SENTINEL2_L1C.define_from(
               "s2l1c", service_url=config.sh_base_url
           ),
           time_interval=("2019-07-01", "2019-07-31"),
           other_args={"dataFilter": {"mosaickingOrder": "leastCC"}}
        ), 
    ],

    responses=[
     
    {
        "identifier": "TrueColor",
        "format": {"type": "image/tiff"},
    },
    {
        "identifier": "B04",
        "format": {"type": "image/tiff"},
    },
    {
        "identifier": "B06",
        "format": {"type": "image/tiff"},
    },
    {
        "identifier": "B08",
        "format": {"type": "image/tiff"},
    },
    {
        "identifier": "B11",
        "format": {"type": "image/tiff"},
    },
    ],
bbox=aoi_bbox,
size=aoi_size,
config=config,
)

In [ ]:
Optional AOI #3:
South East Calabria
(16 38.75, 16 37.75, 17 37.75, 17 38.75, 16 38.75)

# save images

In [ ]:
true_color_imgs = []
request_4_bands[0].get_data()
print(
    f"Returned data is of type = {type(true_color_imgs)} and length {len(true_color_imgs)}."
)
print(
    f"Single element in the list is of type {type(true_color_imgs[-1])} and has shape {true_color_imgs[-1].shape}"
)

In [ ]:
from PIL import Image

true_color_imgs = []
# Ensure the "data" folder exists in the parent directory
output_dir = "../data"
os.makedirs(output_dir, exist_ok=True)

# Loop through each result to fetch the corresponding image
for result in new_results:
    timestamp = result["properties"]["datetime"]
    result_id = result["id"]  # Use the result's ID for naming
    print(f"Fetching image for timestamp: {timestamp}, ID: {result_id}")

    # Request for a single image corresponding to the timestamp
    request_true_color = SentinelHubRequest(
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C.define_from(
                    name="s2l1c", service_url="https://sh.dataspace.copernicus.eu"
                ),
                time_interval=(timestamp, timestamp),  # Use specific timestamp for each result
                other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=aoi_bbox,
        size=aoi_size,
        config=config,
    )

    # Get the data (the image) and append to the list
    image = request_true_color.get_data()
    
    # Since get_data() returns a list of images, take the first one
    true_color_imgs.append(image[0])  # Append the first image (or modify as needed)
    
    # Save the image to the "data" folder
    image_array = image[0]
    image_path = os.path.join(output_dir, f"{result_id}.tiff")  # Use result ID for naming
    Image.fromarray(image_array).save(image_path)
    
    print(f"Fetched and stored image for {timestamp} with ID {result_id} at {image_path}")

# eval double up

In [ ]:

evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B04", "B06", "B08", "B11"]
            }],
            output: {
                bands: 4
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B06, sample.B08, sample.B11];
    }
"""

request_true_color = SentinelHubRequest(
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C.define_from(
                name="s2l1c", service_url="https://sh.dataspace.copernicus.eu"
            ),
            time_interval=("2019-07-01", "2019-07-31"),
            other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=aoi_bbox,
    size=aoi1_size, # exchange for aoi2_size for Corsica , aoi3_size for Calabria as req.
    config=config,
)

# alt auth

In [ ]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

# Your client credentials
client_id = '<client_id>'
client_secret = '<secret>'

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret, include_client_id=True)

# All requests using this session will have an access token automatically added
resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")
print(resp.content)